<a href="https://colab.research.google.com/github/geraldmc/torch-draft-final_project/blob/main/load_deepweeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DeepWeeds TESTING

In [ ]:
import os 
import time
from datetime import datetime
import glob 
import shutil
import copy
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.data import RandomSampler, random_split
from torch.utils.data import SubsetRandomSampler, WeightedRandomSampler
from torchvision.datasets import ImageFolder

### Download code from Github

In [ ]:
if os.path.isfile("../main.zip"):
  print ('Have already downloaded the project file, continuing...')
  print()
else:
  print ('Downloading file...')
  ! wget https://github.com/geraldmc/torch-draft-final_project/archive/refs/heads/main.zip
  ! unzip -qq main.zip
  %cd torch-draft-final_project-main

### Import Github project.

In [ ]:
try:
  import conf.params as params
  from data import transforms as tsf
  from data import test_loader
except ImportError:
  pass

### Download dataset from G Drive.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
print()
print("Downloading DeepWeeds images to " + params.IMG_ZIP_FILE)
!cp '{params.GD_ZIP_IMG}' '{params.IMG_ZIP_FILE}'
print()
!ls -lart {params.IMG_ZIP_FILE}

print()
print("Downloading GAN images to " + params.GAN_ZIP_FILE)
!cp '{params.GD_ZIP_GAN}' '{params.GAN_ZIP_FILE}'
print()
!ls -lart {params.GAN_ZIP_FILE}

### Unzip the data files.

In [ ]:
print("[INFO] Unzipping DeepWeeds images into " +  params.IMG_DIRECTORY)

with ZipFile(params.IMG_ZIP_FILE, "r") as zip_ref:
    zip_ref.extractall(params.IMG_DIRECTORY)

img_list=os.listdir(params.IMG_DIRECTORY)
print(len(img_list))

print()
print("[INFO] Unzipping GAN image dirs into " + params.DATA_PATH)

with ZipFile(params.GAN_ZIP_FILE, "r") as zip_ref:
    zip_ref.extractall(params.DATA_PATH)

gan_dir_list=os.listdir(params.DATA_PATH+'/gans/train/0')
print(len(gan_dir_list))

### Get the labels.

In [ ]:
LABEL_PATH = os.path.join(params.DATA_PATH, 'labels')
label_df = pd.read_csv(os.path.join(LABEL_PATH, 'labels.csv'))
None

## Steps For Test

##### 0) Get files in order (mainly Colab-specific).

    1) Copy files to single directory (not using ImageFolder).
    2) Instantiate data loaders.
    3) Load the trained ResNet50 model.
    4) Test the model. Save the best model.
    5) Delete contents of the test directory.
    7) REPEAT 1-5.

### 1) Supporting functions.


In [ ]:
# 1) Functions to copy test files.

def get_file_list():
    files = []
    for dirpath, dirnames, filenames in os.walk(params.IMAGE_PATH):
        for file in filenames:
            files.append(file)
    return files

def copy_test_files(df, filepath):
  for f in df.Filename:
      try:
          src = os.path.join(params.IMG_DIRECTORY, f)
          #print(src)
          dst = os.path.join(filepath, f)
          shutil.copyfile(src, dst)
          #print (dst)
      except KeyError:
          pass

def delete_test_files(path):
  for file_name in os.listdir(params.IMG_TEST_PATH):
    file = os.path.join(path, file_name)
    if os.path.isfile(file):
      os.remove(file)

# EXECUTE -------
for idx in range(params.FOLDS):
  test_label_file = "{}/test_subset{}.csv".format(LABEL_PATH, idx)
  test_df = pd.read_csv(test_label_file)
  copy_test_files(test_df, params.IMG_TEST_PATH)
  #--------------
  #
  #. DO YOUR THING....
  #
  #--------------
  delete_test_files(params.IMG_TEST_PATH)
  cnt = len([name for name in os.listdir(params.IMG_TEST_PATH) \
             if os.path.isfile(os.path.join(params.IMG_TEST_PATH, name))])
  assert cnt == 0

### Load a model to test.

In [ ]:
def load_model(name):
    model.load_state_dict(torch.load(params.OUTPUT_PATH+name))
    return model
    
def checkpoint(epoch, nn, opt, LOSS, MODEL_PATH ):
    torch.save({
                'epoch': EPOCH,
                'model_state_dict': nn.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': LOSS,
                }, MODEL_PATH)

def print_states(m, opt):
    # Print the model's state_dict
    print("Model's state_dict:")
    for param_tensor in m.state_dict():
        print(param_tensor, "\t", m.state_dict()[param_tensor].size())

    # Print optimizer's state_dict
    print("Optimizer's state_dict:")
    for var_name in opt.state_dict():
        print(var_name, "\t", opt.state_dict()[var_name])

In [ ]:
''' GUTTER ------ 
'''